In [8]:
import os
os.chdir(r'C:\Users\zeeshan.hameed\Desktop\assignment1\assign1TSFD')

In [9]:
%pwd

'C:\\Users\\zeeshan.hameed\\Desktop\\assignment1\\assign1TSFD'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareDataConfig:
    data_path: Path
    train_size : float
    coll_name : str
    look_Back : int

In [24]:
from timeseries.constants import *
from timeseries.utils.common import read_yaml


class DataConfigrationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    def prepare_data(self)->PrepareDataConfig:
        config=self.config.prepare_data
        prepare_base_model_config = PrepareDataConfig(data_path=Path(config.data_path),train_size=self.params.TRAIN_SIZE,
                                                      coll_name=config.col_name,look_Back=self.params.LOOK_BACK)
        return prepare_base_model_config


In [25]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from timeseries.utils.common import create_dataset,transform

class PrepareDataset:
    def __init__(self,config : PrepareDataConfig):
        self.config = config
    

    def dataset_prepration(self):
        datasetpath=self.config.data_path
        column_name = self.config.coll_name
        train_Size=self.config.train_size
        look_back=self.config.look_Back
        df = pd.read_excel(datasetpath ,engine='openpyxl')
        df.Date = pd.to_datetime(df.Date)
        df = df.set_index("Date")
        dataset = df.filter([column_name])
        dataset=transform(dataset)
        train_size = int(len(dataset) * train_Size)
        test_size = len(dataset) - train_size
        train = dataset[0:train_size,:]
        test = dataset[train_size:len(dataset),:]
        Train_X,trainY = create_dataset(train,look_back)
        Test_X,testY = create_dataset(test,look_back)
        trainX = np.reshape(Train_X, (Train_X.shape[0], Train_X.shape[1], 1))
        testX = np.reshape(Test_X, (Test_X.shape[0], Test_X.shape[1], 1))


        return trainX,trainY,testX,testY





In [26]:
config = DataConfigrationManager()
P_data= config.prepare_data()
dataset = PrepareDataset(P_data)
Xtr,Ytr,Xte,Yte = dataset.dataset_prepration()


[2024-02-02 11:00:57,324: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-02 11:00:57,326: INFO: common: yaml file: params.yaml loaded successfully]


In [27]:
Xtr

array([[[0.09527784],
        [0.05477528],
        [0.        ],
        ...,
        [0.65314354],
        [0.72517726],
        [0.87778079]],

       [[0.05477528],
        [0.        ],
        [0.02514864],
        ...,
        [0.72517726],
        [0.87778079],
        [0.72804648]],

       [[0.        ],
        [0.02514864],
        [0.12257942],
        ...,
        [0.87778079],
        [0.72804648],
        [0.80528276]],

       ...,

       [[0.37348614],
        [0.37006923],
        [0.44020197],
        ...,
        [0.62059149],
        [0.52699402],
        [0.5099414 ]],

       [[0.37006923],
        [0.44020197],
        [0.34796902],
        ...,
        [0.52699402],
        [0.5099414 ],
        [0.32451021]],

       [[0.44020197],
        [0.34796902],
        [0.33908694],
        ...,
        [0.5099414 ],
        [0.32451021],
        [0.56900778]]])